In [38]:
import numpy as np
import tensorflow as tf 
import sionna 
from sionna.rt import Scene, Transmitter, RadioMaterial, load_scene, PlanarArray, Receiver

import csv

In [126]:
scene = Scene("../../data/blender/2F_solid.xml")

In [ ]:
scene.preview()

In [44]:
# Add the tranmistters from the data positions file
def load_transmitters(filename):
    transmitters = []
    with open(filename, "r") as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            name = row[0]
            x, y, z = map(float, row[1:])
            transmitters.append((name, [x, y, z])) 
    return transmitters

# Rotate and translate the AP Points 
Make the necessary transformations until they match the correct position

In [45]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
data = pd.read_csv('data/2f.csv', header=None)
data.columns = ['Object', 'X', 'Y', 'Z']
print(data)

In [47]:
def rotation_matrix(theta_x, theta_y, theta_z):
    """
    Create a rotation matrix for the given angles (in radians) around x, y, z axes.
    """
    # Rotation around X
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(theta_x), -np.sin(theta_x)],
                   [0, np.sin(theta_x), np.cos(theta_x)]])
    
    # Rotation around Y
    Ry = np.array([[np.cos(theta_y), 0, np.sin(theta_y)],
                   [0, 1, 0],
                   [-np.sin(theta_y), 0, np.cos(theta_y)]])
    
    # Rotation around Z
    Rz = np.array([[np.cos(theta_z), -np.sin(theta_z), 0],
                   [np.sin(theta_z), np.cos(theta_z), 0],
                   [0, 0, 1]])
    
    # Combined rotation matrix
    return Rz @ Ry @ Rx

def apply_transformation(data, theta_x, theta_y, theta_z, translation):
    """
    Apply rotation and translation to the data points.
    """
    # Create rotation matrix
    R = rotation_matrix(theta_x, theta_y, theta_z)
    
    # Apply transformation
    transformed_data = data.copy()
    
    for i in range(len(data)):
        point = np.array([data.iloc[i]['X'], data.iloc[i]['Y'], data.iloc[i]['Z']])
        rotated_point = R @ point  # Rotate the point
        transformed_data.iloc[i, 1:4] = rotated_point + translation  # Translate the point
    
    return transformed_data


In [48]:
# Define rotation angles in radians
theta_x = np.radians(0)  # Rotation about X axis
theta_y = np.radians(0)  # Rotation about Y axis
theta_z = np.radians(-90)  # Rotation about Z axis

# Define translation vector
translation = np.array([-175, 315, 0])  # Translation along X, Y, Z axes

In [49]:
# Apply the transformations
transformed_data = apply_transformation(data, theta_x, theta_y, theta_z, translation)

In [ ]:
# Plotting the original and transformed points
fig = plt.figure(figsize=(12, 6))

# Original data plot
ax1 = fig.add_subplot(121, projection='3d')
ax1.scatter(data['X'], data['Y'], data['Z'], color='b', label='Original')
ax1.set_title('Original Data')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.legend()

# Transformed data plot
ax2 = fig.add_subplot(122, projection='3d')
ax2.scatter(transformed_data['X'], transformed_data['Y'], transformed_data['Z'], color='r', label='Transformed')
ax2.set_title('Transformed Data')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')
ax2.legend()

plt.show()

In [51]:
for tx in scene.transmitters:
    scene.remove(tx)

In [53]:
transformed_data.to_csv('transformed_data.csv', index=False, header=False)

In [54]:
transmitters = load_transmitters("transformed_data.csv")

for name, position in transmitters:
    tx = Transmitter(name=name, position=position)
    scene.add(tx)

In [55]:
scene.preview()

# Simulate signal

In [20]:
scene.frequency = 2.4e9

In [133]:
# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=2,
                            num_cols=2,
                            vertical_spacing=0.5,    
                            horizontal_spacing=0.5,  
                            pattern="dipole",        
                            polarization="VH")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=2,
                        num_cols=2,
                        vertical_spacing=0.5,
                        horizontal_spacing=0.5,
                        pattern="dipole",
                        polarization="VH")

rx = Receiver(name="rx",
                 position=tf.Variable([20, -10, 15], dtype=tf.float32),
                 orientation=tf.Variable([0, 0, 0], dtype=tf.float32))

scene.add(rx)

In [134]:
tx = Transmitter(name="test", position=[1, 1, 15], orientation=[0,0,0])
scene.add(tx)
tx.look_at(rx)

In [ ]:
print(scene.transmitters)
print(scene.receivers)

In [137]:
paths = scene.compute_paths()

In [ ]:
scene.preview(paths=paths) 

In [143]:
cm = scene.coverage_map(cm_cell_size=[1.,1.], num_samples=int(1e6))

In [ ]:
cm.show()